In [1]:
import os
import re
import pandas as pd

from scipy.stats import spearmanr
from tqdm import tqdm

In [2]:
meta = pd.read_csv("LungCancer10x/03_each_cells/total/Alveolar_II/scanpy/meta.csv.gz", index_col=0)
meta.head()

,nGene,nUMI,SampleID,Batch,Tissue,Cells,batch,percent.mito,res.0.6,cell_name,PatientID,Gender,Disease,Stage,Age,T,N,M,res.0.8,orig.ident
2018jz1_ACAGCCGTCTCTGTCG-1,704,1026,2018jz1,1,Tumor,2018jz1_ACAGCCGTCTCTGTCG-1,First,0,3,Alveolar II,PA13,Male,LUAD,II,73,T2b,N1,M0,5,2018jz1
2018jz1_ACATGGTAGTACGATA-1,3460,17232,2018jz1,1,Tumor,2018jz1_ACATGGTAGTACGATA-1,First,0,4,Alveolar II,PA13,Male,LUAD,II,73,T2b,N1,M0,1,2018jz1
2018jz1_ACGCAGCCAACCGCCA-1,3268,10057,2018jz1,1,Tumor,2018jz1_ACGCAGCCAACCGCCA-1,First,0,21,Alveolar II,PA13,Male,LUAD,II,73,T2b,N1,M0,9,2018jz1
2018jz1_CACCAGGTCTTGTCAT-1,3241,16954,2018jz1,1,Tumor,2018jz1_CACCAGGTCTTGTCAT-1,First,0,3,Alveolar II,PA13,Male,LUAD,II,73,T2b,N1,M0,7,2018jz1
2018jz1_CCGTACTGTTTGTTTC-1,4038,20429,2018jz1,1,Tumor,2018jz1_CCGTACTGTTTGTTTC-1,First,0,21,Alveolar II,PA13,Male,LUAD,II,73,T2b,N1,M0,5,2018jz1


In [ ]:
import os

import pandas as pd
import scanpy as sc
from anndata import AnnData


def load_data(path:str):
    u"""
    load data from specific path
    
    :param path: path to directory
    """
    raw = pd.read_csv(os.path.join(path, "raw.csv.gz"), index_col=0, engine="python")
    scale = pd.read_csv(os.path.join(path, "scale.csv.gz"), index_col=0, engine="python")
    meta = pd.read_csv(os.path.join(path, "meta.csv.gz"), index_col=0, engine="python")
    meta["ident"] = ["{}-{}".format(x["Disease"], x["Stage"]) for _, x in meta.iterrows()]
    
    data = AnnData(X=scale.transpose(), obs = meta)
    data.obs = meta
    data.raw = AnnData(X=raw.transpose(), obs = meta)
    
    return data

obj = load_data("LungCancer10x/03_each_cells/total/Alveolar_II/scanpy")

sc.tl.rank_genes_groups(obj, "ident", use_raw=True, method='wilcoxon')

In [ ]:
markers = {}
for i in ['scores', 'names', 'logfoldchanges', 'pvals', 'pvals_adj']:
    markers[i] = obj.uns["rank_genes_groups"][i]